In [2]:
import pandas as pd 
import numpy as np 
import os
import re
import matplotlib.pyplot as plt
from Bio import SeqIO
from itertools import groupby
import seaborn as sns
currdir = os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)


AnnotatedKinaseSheet = f'../10415/10415_unbiased_normalized_041124_AnnotatedHumanKinome.csv'
PhosphoSheet = f'../10415/10415_phos_normalized_041124.xlsx'

data = pd.read_csv(AnnotatedKinaseSheet, index_col=0)
phospho = pd.read_excel(f'{PhosphoSheet}', sheet_name='Normalized Data')


In [4]:

data['Human_Kinase'] = data['Human_Kinase'].replace('False', False)

kinases = data[data['Human_Kinase'] != False].reset_index(drop=True)

out = list(set(phospho['Accession']) & set(kinases['Accession']))
print(len(out), kinases.shape)

# make non-phospho kinases unique
print(kinases.loc[0, 'Accession'])
if '_k' not in kinases.loc[0, 'Accession'].lower():
    kinases['Accession'] = kinases['Accession'].apply(lambda x: f'{x.split(";")[0]}_k')
kinases.head()

235 (316, 53)
A0A0B4J2F2;P57059


,Accession,Human_Kinase,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,...,112757 Fxn9_Control Normalized,112771 Fxn9_Control Normalized,112750 Fxn9_Rxn Normalized,112764 Fxn9_Rxn Normalized,112778 Fxn9_Rxn Normalized,112780_1 SPQC Normalized,112780_2 SPQC Normalized,112780_3 SPQC Normalized,112780_4 SPQC Normalized,112780_5 SPQC Normalized
0,A0A0B4J2F2_k,SIK1B_HUMAN,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,15,10.12.10.10.11.13.11.5.9.10.7.7.9.5.6.10.9.4.5...,8966.244557,7308.774602,8763.373499,9420.749301,...,10948.819020,10442.664055,8384.168996,7871.567683,8049.957273,4257.345850,4038.235115,3309.965378,3857.857478,4024.544669
1,O00141_k,SGK1_HUMAN,"Serine/threonine-protein kinase Sgk1, org=Homo...",SGK1,7,6.5.6.6.5.4.4.2.4.3.1.4.2.6.4.4.4.5.2.3.3.3.3....,15870.258819,10343.783089,11770.911566,12676.190954,...,4345.729843,3979.979593,5989.902286,6132.303425,4986.231928,13326.720659,14484.834112,14754.400029,14078.329655,15085.227849
2,O00238_k,BMR1B_HUMAN,"Bone morphogenetic protein receptor type-1B, o...",BMPR1B,1,-.-.-.-.-.-.1.-.-.-.-.-.1.-.1.1.1.1.-.-.-.-.-....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.080022,NaN,NaN
3,O00311_k,CDC7_HUMAN,"Cell division cycle 7-related protein kinase, ...",CDC7,30,26.25.24.23.25.20.13.9.13.8.10.13.13.16.12.14....,35625.073555,29760.044580,33417.690964,35242.215973,...,32171.449267,32483.607996,33642.150254,32934.689033,32361.189431,17705.582224,20041.352882,18447.772206,16591.349876,18826.094684
4,O00329_k,PK3CD_HUMAN,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",PIK3CD,13,8.7.6.10.11.8.7.3.5.3.6.6.6.5.7.10.5.6.6.7.6.9...,3381.358547,2288.085433,2117.901021,4732.677458,...,3509.196311,3510.079463,4743.315554,5424.786788,4532.702194,2301.926871,2703.181597,2658.167482,2596.690570,2624.359482


In [5]:
kinases.columns Fxn10_Rxn_CV


Index(['Accession', 'Human_Kinase', 'Description', 'Genes', 'Peptides',
       'Detected_Imputed', '112744 Fxn10_Control Normalized',
       '112758 Fxn10_Control Normalized', '112772 Fxn10_Control Normalized',
       '112751 Fxn10_Rxn Normalized', '112765 Fxn10_Rxn Normalized',
       '112779 Fxn10_Rxn Normalized', '112738 Fxn4_Control Normalized',
       '112752 Fxn4_Control Normalized', '112766 Fxn4_Control Normalized',
       '112745 Fxn4_Rxn Normalized', '112759 Fxn4_Rxn Normalized',
       '112773 Fxn4_Rxn Normalized', '112739 Fxn5_Control Normalized',
       '112753 Fxn5_Control Normalized', '112767 Fxn5_Control Normalized',
       '112746 Fxn5_Rxn Normalized', '112760 Fxn5_Rxn Normalized',
       '112774 Fxn5_Rxn Normalized', '112740 Fxn6_Control Normalized',
       '112754 Fxn6_Control Normalized', '112768 Fxn6_Control Normalized',
       '112747 Fxn6_Rxn Normalized', '112761 Fxn6_Rxn Normalized',
       '112775 Fxn6_Rxn Normalized', '112741 Fxn7_Control Normalized',
       '1

In [6]:
all = pd.concat([kinases, phospho], axis=0)
all

,Accession,Human_Kinase,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A0B4J2F2_k,SIK1B_HUMAN,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,15.0,10.12.10.10.11.13.11.5.9.10.7.7.9.5.6.10.9.4.5...,8966.244557,7308.774602,8763.373499,9420.749301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O00141_k,SGK1_HUMAN,"Serine/threonine-protein kinase Sgk1, org=Homo...",SGK1,7.0,6.5.6.6.5.4.4.2.4.3.1.4.2.6.4.4.4.5.2.3.3.3.3....,15870.258819,10343.783089,11770.911566,12676.190954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O00238_k,BMR1B_HUMAN,"Bone morphogenetic protein receptor type-1B, o...",BMPR1B,1.0,-.-.-.-.-.-.1.-.-.-.-.-.1.-.1.1.1.1.-.-.-.-.-....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,O00311_k,CDC7_HUMAN,"Cell division cycle 7-related protein kinase, ...",CDC7,30.0,26.25.24.23.25.20.13.9.13.8.10.13.13.16.12.14....,35625.073555,29760.044580,33417.690964,35242.215973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O00329_k,PK3CD_HUMAN,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",PIK3CD,13.0,8.7.6.10.11.8.7.3.5.3.6.6.6.5.7.10.5.6.6.7.6.9...,3381.358547,2288.085433,2117.901021,4732.677458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39167,Q9Y6Y8,NaN,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,NaN,1.1.-.1.1.2.1.1.-.1.1.1.1.2.-.1.2.-.2.1.1.2.-....,90.819400,114.393484,66.188372,235.425495,...,23.9,27.4,24.4,39.9,36.7,49.3,18.9,34.3,22.3,25.1
39168,TRYP_PIG,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1....,7.512516,5.905145,3.663518,1.021589,...,46.9,32.4,31.2,34.9,85.8,81.0,30.3,161.0,152.0,78.2
39169,TRYP_PIG,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,-.1.-.-.1.-.1.1.-.1.1.1.-.1.1.-.1.1.-.1.1.1.1....,624.474396,896.785816,2216.152687,1.085434,...,40.6,45.1,66.9,87.8,54.2,31.6,97.7,47.3,85.7,46.2
39170,TRYP_PIG,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,-.1.-.-.1.-.1.-.-.1.-.-.-.1.-.1.1.1.1.1.1.-.-....,9.375021,6.000127,3.592862,2.411671,...,48.5,29.6,29.4,50.5,32.2,50.6,24.2,47.8,61.1,62.0
